In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#This Cell Block is all the functions for the strat. This is the only cell block that needs to be changed to backtest a different strategy.

def strat(ticker, period):   
    #Import your strat code here

    data = yf.download(ticker, period = period, interval = "1d")

    short_window = 7  # Short-term moving average window
    long_window = 60   # Long-term moving average window

    data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()
    
    data['Signal'] = 0
    data['Signal'][data['Short_MA'] > data['Long_MA']] = 1  # Buy signal
    data['Signal'][data['Short_MA'] < data['Long_MA']] = -1  # Sell signal

    data['Position'] = data['Signal'].shift(1)

    # Calculate daily returns
    data['Daily Return'] = data['Close'].pct_change()
    # Calculate strategy returns: Position * Daily Return
    data['Strategy Return'] = data['Position'] * data['Daily Return']
    
    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod() - 1
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod() - 1

    total_strategy_return = data['Cumulative Strategy Return'].iloc[-1] * 100
    total_strategy_return = "{:.2f}%".format(total_strategy_return)
    
    return total_strategy_return        

In [3]:
#The backtesting function to backtest the strat over different periods
def backtest_strat(ticker):
    backtest_data = {
        'Asset': [ticker],
        'YTD': [strat(ticker, "ytd")],
        'Whole Period': [strat(ticker, "max")],
        'Last 10Y': [strat(ticker, "10y")],
        'Last 5Y': [strat(ticker, "5y")],
        'Last 2Y': [strat(ticker, "2y")],
        'Last 1Y': [strat(ticker, "1y")],
        'Last 6M': [strat(ticker, "6mo")],
        'Last 3M': [strat(ticker, "3mo")],
        'Last 1M': [strat(ticker, "1mo")]    }
    
    # Create a pandas dataframe
    backtest_df = pd.DataFrame(backtest_data)
    
    # Set the first column as the index for clarity
    backtest_df.set_index('Asset', inplace=True)
    return backtest_df

In [4]:
#Creates an empty dataframe to store the backtest data
df =pd.DataFrame({
        'Asset': [],'YTD': [], 'Whole Period': [], 'Last 10Y': [], 'Last 5Y': [], 'Last 2Y': [], 'Last 1Y': [], 'Last 6M': [], 'Last 3M': [], 'Last 1M': []
    })

df.set_index('Asset', inplace=True)

In [5]:
#Fetching the tickers for the S&P 500
#This will take like 10mins to execute
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
symbols = [stock['symbol'] for stock in sp500_symbols]

#Test Symbols
#symbols = ["TSLA", "NVDA", "AAPL", "MSFT", "MSTR", "AMZN", "META", "INTC"]

#Running the stocks through the strat and backtesting functions
for symbol in symbols:
    try:
        df = pd.concat([df, backtest_strat(symbol)])
    except IndexError:
        pass

df

KeyboardInterrupt: 

In [7]:
#Saves the dataframe to a csv file. Edit the name
df.to_csv('Moving Averages short 7 long 60.csv', index=True)